# Duplication and coherence

## Overall purpose and objective
The overall purpose and objective of the cleaning and verification process is to prepare the data for conversion into a SQLite database (Datasette). As such, the data should follow database best practices.

## Specific purpose of this notebook
This notebook is for checking duplicates in the data and coherence. Particularly, we want to check for:
- Possible problematic rows for each table.
- The companies/agencies in the company and agency lists correspond to companies/agencies in the company data

## Assumptions
- Some combinations of fields should be unique
- The values should be cohrent across tables

## Why this matters 
- Inserting the data in a proper database and assigning EITI IDs require a high confidence in the data quality to avoid downstream issues. Duplicates and non-coherence of the data decreases this level of confidence.

## Findings


## Analysis

In [1]:
# import libraries and data

import pandas as pd
import numpy as np
from os import path
from functools import reduce
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from itertools import combinations

file_dir = "data/consolidated/"

# load the csvs into data frames
df_part_1 = pd.read_csv(path.join(file_dir, "Part 1 - About.csv"))
df_part_3a = pd.read_csv(path.join(file_dir, "Part 3 - Reporting companies' list.csv"))
df_part_3b = pd.read_csv(path.join(file_dir, "Part 3 - Reporting government entities list.csv"))
df_part_3c = pd.read_csv(path.join(file_dir, "Part 3 - Reporting projects' list.csv"))
df_part_4 = pd.read_csv(path.join(file_dir, "Part 4 - Government revenues.csv"))
df_part_5 = pd.read_csv(path.join(file_dir, "Part 5 - Company data.csv"))
# df_part_5 = pd.read_csv(path.join(file_dir, "Part 5 - Company data.csv"), low_memory=False)

df_list = [df_part_1, df_part_3a, df_part_3b, df_part_3c, df_part_4, df_part_5]
df_dict = {"Part 1 - About.csv": df_part_1,
           "Part 3 - Reporting companies' list.csv": df_part_3a,
           "Part 3 - Reporting government entities list.csv": df_part_3b,
           "Part 3 - Reporting projects' list.csv": df_part_3c,
           "Part 4 - Government revenues.csv": df_part_4,
           "Part 5 - Company data.csv": df_part_5
          }

# OPTIONAL COLUMNS
part_3a_opt = ["Stock exchange listing or company website", 
               "Audited financial statement (or balance sheet, cash flows, profit/loss statement if unavailable)"]
part_3b_opt = ["ID number (if applicable)"]
part_5_opt = ["In-kind volume (if applicable)", "Unit (if applicable)", "Comments"]

# only include fields that are non-optional
df_part_1_non_opt = df_part_1.copy()
df_part_3a_non_opt = df_part_3a.copy().drop(columns=part_3a_opt)               
df_part_3b_non_opt = df_part_3b.copy().drop(columns=part_3b_opt)
df_part_3c_non_opt = df_part_3c.copy()
df_part_4_non_opt = df_part_4.copy()
df_part_5_non_opt = df_part_5.copy().drop(columns=part_5_opt)

df_list_non_opt = [df_part_1_non_opt, df_part_3a_non_opt, df_part_3b_non_opt, df_part_3c_non_opt, df_part_4_non_opt, df_part_5_non_opt]
df_dict_non_opt = {"Part 1 - About.csv": df_part_1_non_opt,
           "Part 3 - Reporting companies' list.csv": df_part_3a_non_opt,
           "Part 3 - Reporting government entities list.csv": df_part_3b_non_opt,
           "Part 3 - Reporting projects' list.csv": df_part_3c_non_opt,
           "Part 4 - Government revenues.csv": df_part_4_non_opt,
           "Part 5 - Company data.csv": df_part_5_non_opt
          }

In [2]:
def get_column_combinations(columns, min_cols_diff):
    '''
    Get unique column combinations that will be used for determining problematic rows (i.e. possible duplicates).

    Parameters:
    - columns (list or iterable): A list of column names or identifiers to be used for forming combinations.
    - min_cols_diff (int): The minimum number of columns to form unique combinations.

    Returns:
    list: A list of tuples representing unique combinations of columns.

    Example:
    >>> columns = ['col1', 'col2', 'col3']
    >>> min_cols_diff = 2
    >>> get_column_combinations(columns, min_cols_diff)
    [('col1', 'col2'), ('col1', 'col3'), ('col2', 'col3')]
    '''

    all_combinations = []
    for x in range(min_cols_diff, len(columns) + 1):
        all_combinations.extend(combinations(columns, x))

    return all_combinations

# TEST: Uncomment lines below and run
# OUTPUT: 4 unique column combinations
# columns = ["Full name of agency", "Agency type", "Total reported"]
# column_combinations = get_column_combinations(columns, 2)
# print(f'There are {len(column_combinations)} combinations:')
# pprint(column_combinations)

In [3]:
def add_rowid(df):
    '''
    Add a row identifier (rowid) column to the DataFrame.

    Parameters:
    - df (pandas.DataFrame): The input DataFrame.

    Returns:
    pandas.DataFrame: A new DataFrame with an additional 'rowid' column.

    Example:
    >>> data = {'col1': [1, 2, 3], 'col2': ['a', 'b', 'c']}
    >>> df = pd.DataFrame(data)
    >>> add_rowid(df)
       col1 col2  rowid
    0     1    a      0
    1     2    b      1
    2     3    c      2
    '''

    df_rowid = df.copy()
    df_rowid["rowid"] = range(len(df_rowid))

    return df_rowid    

In [4]:
def get_problematic_rows(df, columns, min_cols_diff):
    '''
    Get unique column combinations and find problematic rows (duplicates) based on specified columns.

    Parameters:
    - df (pandas.DataFrame): The input DataFrame.
    - columns (list or iterable): A list of column names or identifiers to be used for forming combinations.
    - min_cols_diff (int): The minimum number of columns to form unique combinations (i.e. minimum numbers of columns of difference to be considered unique).

    Returns:
    pandas.DataFrame: DataFrame containing unique rows among problematic rows.

    Example:
    >>> data = {'col1': [1, 2, 2, 3, 4], 'col2': ['a', 'b', 'b', 'c', 'd']}
    >>> df = pd.DataFrame(data)
    >>> columns_for_combinations = ['col1', 'col2']
    >>> min_cols_diff = 2
    >>> get_problematic_rows(df, columns_for_combinations, min_cols_diff)
       col1 col2
    0     2    b
    '''

    # Step 1: Create a copy of the DataFrame and add rowid
    df_copy = df.copy()
    df_copy['rowid'] = range(len(df_copy))

    # Step 2: Get column combinations
    all_combinations = []
    for x in range(min_cols_diff, len(columns) + 1):
        all_combinations.extend(combinations(columns, x))

    # Step 3: Find problematic rows for each column combination
    problematic_rows = pd.DataFrame()
    for combination in all_combinations:
        duplicated_rows = df_copy[df_copy.duplicated(subset=list(combination), keep=False)]
        problematic_rows = pd.concat([problematic_rows, duplicated_rows], ignore_index=False)

    # Step 4: Get unique rows among problematic rows
    unique_problematic_rows = problematic_rows.drop_duplicates()

    return unique_problematic_rows

In [5]:
# SAMPLE TEST
sample = pd.DataFrame({
    "Full name of agency": ["A", "A", "B", "B", "C", "D", "E"],
    "Agency type": ["pri", "pri", "pri", "soe", "soe", "soe", "soe"],
    "Total reported": [0, 0, 1, 1, 2, 2, 5],
    "RowID": [0, 1, 2, 3, 4, 5, 6]
})

columns = ["Full name of agency", "Agency type", "Total reported"]

get_problematic_rows(sample, columns, 2)

,Full name of agency,Agency type,Total reported,RowID,rowid
0,A,pri,0,0,0
1,A,pri,0,1,1
2,B,pri,1,2,2
3,B,soe,1,3,3
4,C,soe,2,4,4
5,D,soe,2,5,5


### Part 3 - Reporting government entities list

In [6]:
columns = ["Full name of agency", "Agency type", "Total reported"]

get_problematic_rows(df_part_3b, columns, 2)

,Full name of agency,Agency type,ID number (if applicable),Total reported,Country,ISO Code,Year,Start Date,End Date,rowid
0,Ministry of Finance (Revenue Department),Central goverment,Not applicable,#ERROR!,Afghanistan,AFG,2018.0,2017-12-21,2018-12-20,0
1,Ministry of Finance (Customs Department),Central goverment,Not applicable,#ERROR!,Afghanistan,AFG,2018.0,2017-12-21,2018-12-21,1
2,Ministry of Mines and Petroleum (Revenue Depar...,Central goverment,Not applicable,#ERROR!,Afghanistan,AFG,2018.0,2017-12-21,2018-12-22,2
3,National Environmental Protection Agency,Central goverment,Not applicable,#ERROR!,Afghanistan,AFG,2018.0,2017-12-21,2018-12-23,3
4,Ministry of Industry and Commerce,Central goverment,Not applicable,#ERROR!,Afghanistan,AFG,2018.0,2017-12-21,2018-12-24,4
...,...,...,...,...,...,...,...,...,...,...
479,Other Govt. Agency,Other,NaN,NaN,Tanzania,TZA,2018.0,2017-07-01,2018-06-30,479
488,Les delegations speciales des communes et pref...,Local government,Not applicable,NaN,Togo,TGO,2017.0,2017-01-01,2017-12-31,488
493,Agence Nationale de Gestion de l'Environnement...,Central government,NaN,NaN,Togo,TGO,2018.0,2018-01-01,2018-12-31,493
495,Togolaise des Eaux (TdE),Central government,NaN,NaN,Togo,TGO,2018.0,2018-01-01,2018-12-31,495
